<a href="https://colab.research.google.com/github/thissop/DROPTABLE2022/blob/main/MAXI-J1535/code/notebooks/December-%202021-2022/regressor_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

## Hyperparameters

* SVM Regressor
    * "One of the commonly used kernel functions is radial basis function (RBF)"
    * C: "between 0.1 < c < 100"; I'll do C ⊂ ```{0.1,1,10}``` (default is 1)
        * "If c is small, the penalty for misclassified points is low so a decision boundary with a large margin is chosen at the expense of a greater number of misclassifications"
    * Gamma: how tight the regions are...will use default ```scale```
* RF Regressor
    * Will use default ```squared_error``` for criterion
* Histogram Gradient Boosted Regressor
    * Set ```early_stopping``` to False

## Misc. 
### Helpful Snippets
* ```.best_params_``` returns best fit params from a search. See below: 
    ```python3  
    random_search = RandomizedSearchCV(estimator = ...)
    random_search.fit(...)
    best_combination_dict = random_search.best_params_
    ```

    * Probably going to write my own version of this for better train test splitting...do train test splitting outside of loop. This loop needs to be as efficient as possible.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale as normalize
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, median_absolute_error
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, RandomForestClassifier
from sklearn.multioutput import RegressorChain
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


plt.style.use('seaborn-darkgrid')
plt.rcParams['font.family']='serif'
plt.rcParams['figure.dpi']=150

data_df = pd.read_csv('https://raw.githubusercontent.com/thissop/MAXI-J1535/main/data/processed/fixed_merged.csv?token=AQN5JT7ZW3ZKV5EKEEUCVITB3RP4Y')

In [ ]:
approaches_dict = {'svm':{'approach':'drop', 'hyperparameters':{'C':[0.1,1,10]}},
                   'rf':{'approach':'flag', 'stat_boundaries':[0.25,5], 'hyperparameters':{'min_samples_leaf':[1, 2, 4],'min_samples_split':[2, 5, 10]}}, 
                   'hgbr':{'approach':'missing', 'stat_boundaries':[0.25,5]}, 'hyperparameters':{'learning_rate':[0,1,1,10], 'min_samples_leaf':[1, 2, 4], 'l2_regularization':[0,1]}}

approaches_dict